In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'data/sl/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path = DATA_DIR/'/home/pablo/dev23/data_sets/sl.zip'
gk.list_feed(path)

,file_name,file_size
0,stop_times.txt,132092280
1,calendar.txt,27620
2,agency.txt,1278
3,stops.txt,1434719
4,calendar_dates.txt,676779
5,feed_info.txt,144
6,routes.txt,30622
7,attributions.txt,2279559
8,trips.txt,4804496
9,transfers.txt,1287404


In [12]:
feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Upplands lokaltrafik, Kommuner i Stockholms L..."
1,timezone,Europe/Stockholm
2,start_date,20240212
3,end_date,20241230
4,num_routes,610
5,num_trips,77417
6,num_stops,21568
7,num_shapes,3211
8,sample_date,20240215
9,num_routes_active_on_sample_date,575


In [13]:
feed.validate()

,type,message,table,rows
1,error,Invalid route_type; maybe has extra space char...,routes,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
0,warning,Unrecognized column feed_id,feed_info,[]
2,warning,"Repeated pair (route_short_name, route_long_name)",routes,"[478, 480, 481, 607, 608]"
5,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[1548659, 1548666, 1548673, 1548680, 1548687, ..."
3,warning,Unrecognized column platform_code,stops,[]
4,warning,Stop has no stop times,stops,"[7269, 7298, 7299, 7304, 7307, 7313, 7317, 731..."
6,warning,Unrecognized column from_trip_id,transfers,[]
7,warning,Unrecognized column to_trip_id,transfers,[]


In [14]:
# Append shape_dist_traveled column to stop times
display(feed.stop_times.head().T)
feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

,0,1,2,3,4
trip_id,14010000635912911,14010000635912911,14010000635912911,14010000635912911,14010000635912911
arrival_time,05:53:00,05:53:56,05:54:23,05:54:57,05:55:45
departure_time,05:53:00,05:53:56,05:54:23,05:54:57,05:55:45
stop_id,9022001010028003,9022001010052003,9022001010054002,9022001010056002,9022001010058002
stop_sequence,1,2,3,4,5
stop_headsign,Stora Essingen,Stora Essingen,Stora Essingen,Stora Essingen,Stora Essingen
pickup_type,3,3,3,3,3
drop_off_type,1,3,3,3,3
shape_dist_traveled,0.0,398.26,570.67,808.54,1107.9
timepoint,1,0,0,0,0


/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pablo/anaconda3/envs/geo_env/

,0,1,2,3,4
trip_id,14010000331224227,14010000331224227,14010000331224227,14010000331224227,14010000331224227
arrival_time,06:39:00,06:40:00,06:42:00,06:44:00,06:45:00
departure_time,06:39:00,06:40:00,06:42:00,06:44:00,06:45:00
stop_id,9022001004201001,9022001004411001,9022001004421001,9022001004431001,9022001004441001
stop_sequence,1,2,3,4,5
stop_headsign,Nockeby,Nockeby,Nockeby,Nockeby,Nockeby
pickup_type,3,3,3,3,3
drop_off_type,1,3,3,3,3
timepoint,1,1,1,1,1
shape_dist_traveled,0.0,0.864388,1.329668,2.189957,2.741705


In [15]:
week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday

In [16]:
# Compute feed time series
trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2024-02-16 00:00:00,2289.0,2969.0,2951.0,42154.185621,1339.716667,31.465001
2024-02-16 06:00:00,12186.0,12375.0,13030.0,176824.098790,6738.116667,26.242362
2024-02-16 12:00:00,12408.0,12850.0,13678.0,181212.670979,7181.166667,25.234433
2024-02-16 18:00:00,8216.0,7243.0,8543.0,112408.031917,3964.733333,28.351978
2024-02-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-18 00:00:00,1048.0,1242.0,1235.0,20864.200275,670.933333,31.097278


In [17]:
gk.downsample(fts, freq='12H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2024-02-16 00:00:00,14475.0,15344.0,15319.0,218978.284411,8077.833333,27.108542
2024-02-16 12:00:00,20624.0,20093.0,20951.0,293620.702895,11145.900000,26.343382
2024-02-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-18 00:00:00,6494.0,7149.0,7118.0,102993.386566,3523.683333,29.228900
2024-02-18 12:00:00,13705.0,13367.0,14009.0,195009.526835,7126.316667,27.364701


In [18]:
# Compute feed stats for first week
feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,11787,574,34644,34644,33431,501028.682063,18856.462500,26.570661,1610,07:51:00,07:51:15,20240212
0,11787,576,34659,34659,33446,501208.768805,18864.979167,26.568212,1611,07:51:00,07:51:15,20240213
0,11787,574,34663,34663,33450,501264.142286,18868.095833,26.566758,1611,07:51:00,07:51:15,20240214
0,11787,575,34661,34661,33448,501346.539147,18869.812500,26.568708,1611,07:51:00,07:51:15,20240215
0,11787,578,35437,35437,33454,512598.987307,19224.395833,26.663984,1608,07:51:00,07:51:15,20240216
0,11787,415,21877,21877,19902,315664.124694,11311.237500,27.907126,762,15:05:15,15:05:30,20240217
0,11787,415,20516,20516,19312,298002.913401,10650.583333,27.979962,743,15:40:45,15:41:00,20240218


In [19]:
# Compute route time series
rts = feed.compute_route_time_series(trip_stats, dates, freq='12H')
rts

indicator              num_trip_ends                                    \
route_id            9011001000100000 9011001000200000 9011001000300000   
datetime                                                                 
2024-02-16 00:00:00            101.0             95.0            100.0   
2024-02-16 12:00:00            162.0            146.0            145.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             56.0             60.0             63.0   
2024-02-18 12:00:00            125.0            122.0            122.0   

indicator                                                               \
route_id            9011001000400000 9011001000600000 9011001000700000   
datetime                                                                 
2024-02-16 00:00:00            115.0             81.0             84.0   
2024-02-16 12:00:00            198.0            135.0            158.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             65.0             38.0             60.0   
2024-02-18 12:00:00            153.0             96.0            159.0   

indicator                                                               \
route_id            9011001001000000 9011001001100000 9011001001200000   
datetime                                                                 
2024-02-16 00:00:00            117.0            118.0             89.0   
2024-02-16 12:00:00            154.0            153.0            138.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             65.0             64.0             36.0   
2024-02-18 12:00:00            134.0            133.0             70.0   

indicator                             ...    service_speed                   \
route_id            9011001001300000  ... 9011008002000000 9011008002100000   
datetime                              ...                                     
2024-02-16 00:00:00            143.0  ...        18.633186        17.627850   
2024-02-16 12:00:00            191.0  ...        19.058579        16.698539   
2024-02-17 00:00:00              NaN  ...              NaN              NaN   
2024-02-17 12:00:00              NaN  ...              NaN              NaN   
2024-02-18 00:00:00             60.0  ...        18.894262        15.889901   
2024-02-18 12:00:00            138.0  ...        17.507598        16.369555   

indicator                                                               \
route_id            9011008002200000 9011008002300000 9011008002500000   
datetime                                                                 
2024-02-16 00:00:00        14.500351        11.487748         0.000000   
2024-02-16 12:00:00        14.496769        11.041190        14.667866   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00         0.000000        10.156436         0.000000   
2024-02-18 12:00:00        14.497127        11.280067        12.834382   

indicator                                                               \
route_id            9011008002900000 9011008003000000 9011008003100000   
datetime                                                                 
2024-02-16 00:00:00        10.653874        11.449547        16.831950   
2024-02-16 12:00:00        10.653874        11.449547        17.785629   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00        10.653874        11.449547        18.938285   
2024-02-18 12:00:00        1

In [20]:
# Slice time series
inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '110N-423'] # [478, 480, 481, 607, 608]
rts.loc[:, (inds, rids)]

/tmp/ipykernel_29020/2324798737.py:4: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  rts.loc[:, (inds, rids)]


datetime
2024-02-16 00:00:00
2024-02-16 12:00:00
2024-02-17 00:00:00
2024-02-17 12:00:00
2024-02-18 00:00:00
2024-02-18 12:00:00


In [21]:
# Slice again by cross-section
rts.xs(rids[0], axis="columns", level=1)

KeyError: '110-423'

In [3]:
# Compute trip locations for every hour
rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

NameError: name 'feed' is not defined

In [4]:
# Build a route timetable
route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

NameError: name 'feed' is not defined

In [5]:
# Compute screen line counts
path = DATA_DIR/'cairns_screen_lines.geojson'
lines = gp.read_file(path)
display(lines)
feed.compute_screen_line_counts(lines, dates)

DriverError: ../data/sl/cairns_screen_lines.geojson: No such file or directory

In [6]:
# Map routes
rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids, include_stops=True)

NameError: name 'feed' is not defined